In [1]:
# Install the latest versions of necessary libraries
!pip install -U transformers accelerate bitsandbytes qwen-vl-utils

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 109.0 MB/s eta 0:00:0000:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 34.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 51.3 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.1
    Uninstalling transformers-4.57.1:
      Successfully uninstalled transformers-4.57.1
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.11.0
    Uninstalling accelerate-1.11.0:
      Successfully uninstalled accelerate-1.11.0


In [2]:
import warnings

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=DeprecationWarning)
    # code that triggers noisy deprecation warnings


In [3]:
import torch
from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor
from qwen_vl_utils import process_vision_info

# Load the model with 4-bit quantization to save memory
model_name = "Qwen/Qwen2.5-VL-3B-Instruct"

model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto",
    # trust_remote_code=True, # Uncomment if using older Qwen-VL versions
)

# Load the processor
processor = AutoProcessor.from_pretrained(model_name)

print(f"Model {model_name} loaded successfully!")


2026-01-16 11:58:40.115538: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768564720.295833      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768564720.349204      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768564720.793514      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768564720.793555      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768564720.793558      55 computation_placer.cc:177] computation placer alr

config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.53G [00:00<?, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 93dd84cc-0477-41db-a4d1-b88627d82457)')' thrown while requesting HEAD https://huggingface.co/Qwen/Qwen2.5-VL-3B-Instruct/resolve/main/processor_config.json
Retrying in 1s [Retry 1/5].


chat_template.json: 0.00B [00:00, ?B/s]

Model Qwen/Qwen2.5-VL-3B-Instruct loaded successfully!


In [4]:
image = "/kaggle/input/toys-dataset/toys dataset/Ball_Launcher/image_012.jpg"

In [5]:
# 1. Prepare the input: An image and a question
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image},
            {"type": "text", "text": "Describe this image in detail. and also detect the toy and describe it"},
        ],
    }
]

# 2. Preprocess the inputs
text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True )
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

# 3. Generate the response
generated_ids = model.generate(**inputs, max_new_tokens=512)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)

print("-" * 30)
print("Model Response:")
print(output_text[0])


------------------------------
Model Response:
The image depicts a young girl playing with her dog in an outdoor setting, likely a park or a backyard. The girl is wearing a light blue shirt and light blue shorts, and she has a headband on. She is holding a green frisbee in her right hand, which is extended towards the dog.

The dog, a small breed with a white coat and brown patches, is standing on its hind legs, reaching up to catch the frisbee. The dog's front paws are off the ground, and it appears to be focused on the frisbee. The background shows a grassy area with some trees and bushes, indicating that the location is likely a natural, open space.

The toy in the image is a green frisbee. It is round and has a slightly glossy surface, reflecting some light. The frisbee is being held by the girl, and the dog is attempting to catch it. The frisbee's position suggests that it is in mid-air, just above the dog's reach.
